In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import numpy as np
import torchsummary
import os
from PIL import Image
from tqdm.auto import tqdm

torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3070 Ti'

In [29]:
!pip install tqdm

  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)



[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
# 모델정의
class ResNet50Classifier(nn.Module):
    def __init__(self, num_classes=4800):
        super(ResNet50Classifier, self).__init__()
        
        # Resnet50 model
        # pretrained 모델 사용
        self.backborn = models.resnet50(pretrained=True)
        
        for param in self.backborn.parameters():
            param.requires_grad = False
        
        # resnet50 출력채널수
        num_features = self.backborn.fc.in_features
        
        # resnet50의 마지막 출력채널을 제거
        self.backborn.fc = nn.Identity()
        
        # 우리가 분류할 class만큼 full connected 레이어 추가
        self.intermediate = nn.Linear(num_features, num_classes//2)
        self.classifier = nn.Linear(num_classes//2, num_classes)
    
    def forward(self, x):
        x = self.backborn(x)
        x = self.intermediate(x)
        x = self.classifier(x)
        return x

In [7]:
# 이미지 투명도 제거 Transform
class RemoveAlpha:
    def __call__(self, img):
        img = img.convert('RGB')
        return img

batch_size = 256
    
# 데이터 로더
base_path = "e:\\pill_image_cropped" # './sample_data'

# 데이터셋 전처리 이미 되어있음
transform = transforms.Compose([
#     RemoveAlpha(),
#     transforms.CenterCrop(1200),
#     transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    # transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
])

image_dataset = ImageFolder(base_path, transform=transform)
print(len(image_dataset.classes))

val_size = int(len(image_dataset) * 0.2)
train_size = len(image_dataset) - val_size

train_dataset, val_dataset = torch.utils.data.random_split(image_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

5087


In [11]:
# 학습함수
def train(model, train_loader, valid_loader, criterion, optimizer, device, epochs):
    result = []
    model.to(device)
    print(torchsummary.summary(model, (3, 224, 224)))
    # epochs 만큼 반복
    for epoch in tqdm(range(epochs)):
        # 캐시 비우기
        torch.cuda.empty_cache()
        # train모드
        model.train()
        # train 데이터 가져옴
        for images, labels in tqdm(train_loader, leave=False):
            # 장치로 보냄
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            
            # loss 계산
            loss = criterion(outputs, labels)
            
            # backpropagation
            loss.backward()
            optimizer.step()
            del images, labels
        # validation
        valid_loss = 0.0
        valid_accuracy = 0.0
        # eval모드
        model.eval()
        with torch.no_grad():
            for images, labels in valid_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                
                _, pred = torch.max(outputs, 1)
                accuracy = torch.sum(pred == labels.data)
                
                valid_loss += loss.item() * images.size(0)
                valid_accuracy += accuracy.item()
        valid_loss /= len(valid_loader.dataset)
        valid_accuracy /= len(valid_loader.dataset)
        
        print(f'Epoch {epoch+1}/{epochs} : loss : {loss.item():.3f}, valid_loss : {valid_loss:.3f}, valid_accuracy : {valid_accuracy:.3f}')
        result.append((loss.item(), valid_loss, valid_accuracy))
    return result

In [12]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

model = ResNet50Classifier(num_classes=5087)
# train_loader = 
# valid_loader = None
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
epochs = 12

result = train(model, train_loader, val_loader, criterion, optimizer, device, epochs)

torch.save(model, "0509_12e_model.pt")

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1,

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/645 [00:00<?, ?it/s]

Epoch 1/12 : loss : 1.623, valid_loss : 1.623, valid_accuracy : 0.677


  0%|          | 0/645 [00:00<?, ?it/s]

...

Epoch 10/12 : loss : 0.196, valid_loss : 0.196, valid_accuracy : 0.855


  0%|          | 0/645 [00:00<?, ?it/s]

Epoch 11/12 : loss : 1.045, valid_loss : 1.045, valid_accuracy : 0.859


  0%|          | 0/645 [00:00<?, ?it/s]

Epoch 12/12 : loss : 1.032, valid_loss : 1.032, valid_accuracy : 0.845


In [14]:
# 메모리 정리
# del model
torch.cuda.empty_cache()
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   1168 MiB |   3869 MiB | 345072 GiB | 345071 GiB |
|       from large pool |   1133 MiB |   3834 MiB | 345064 GiB | 345063 GiB |
|       from small pool |     35 MiB |     37 MiB |      7 GiB |      7 GiB |
|---------------------------------------------------------------------------|
| Active memory         |   1168 MiB |   3869 MiB | 345072 GiB | 345071 GiB |
|       from large pool |   1133 MiB |   3834 MiB | 345064 GiB |